In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import holodeck as holo
import holodeck.plot
import holodeck.utils
from holodeck.constants import YR

# SAM Populations

In [ ]:
fname_real = "./pop-sam_v0.2_b54efb6__d20.0_c0.100_sh0050_st+2.000/0000.npz"
fname_sam = "./pop-sam_v0.2_b54efb6__d20.0_c0.100_sh0050_st+2.000/sam.npz"

data_sam = np.load(fname_sam)
fobs = data_sam['fobs']
print(f"SAM frequencies: [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr, {fobs.size-1} bins")

data_real = np.load(fname_real)
weights = data_real['weights']
print(f"Sample points: {weights.size:.4e}, effective number of binaries: {weights.sum():.4e}")

In [ ]:
nloud = 5

hs, fo = holo.sam._strains_from_samples([data_real[kk] for kk in ['mtot', 'mrat', 'redz', 'fobs']])
colors = holo.plot._get_cmap('plasma')(np.linspace(0.1, 0.9, nloud))

fig, ax = holo.plot.figax(figsize=[12, 8], xlabel='Frequency [yr$^{-1}$]', ylabel='c-Strain')
for ii in holo.utils.tqdm(range(fobs.size-1)):
    fextr = [fobs[ii+jj] for jj in range(2)]
    fextr = np.asarray(fextr)
    cycles = 1.0 / np.diff(np.log(fextr))[0]

    idx = (fextr[0] <= fo) & (fo < fextr[1])
    hs_bin = hs[idx]
    fo_bin = fo[idx]    
    ww_bin = weights[idx]
    ww = ww_bin * cycles

    tot = np.sqrt(np.sum(ww * hs_bin**2))
    ax.plot(fextr*YR, tot * np.ones_like(fextr), 'k--')

    idx = np.argsort(hs_bin)[::-1]
    if any(ww_bin[idx[:nloud]] > 1):
        raise
    
    for jj, cc in enumerate(colors):
        if jj > len(idx):
            break
        hi = idx[jj]
        lo = idx[jj+1:]
        gw_hi = np.sqrt(np.sum(ww[hi] * hs_bin[hi]**2))
        gw_lo = np.sqrt(np.sum(ww[lo] * hs_bin[lo]**2))

        fave = np.average(fo_bin[hi], weights=hs_bin[hi])
        ax.plot(fextr*YR, gw_lo * np.ones_like(fextr), color=cc, lw=0.5)
        ax.scatter(fave*YR, gw_hi, marker='.', color=cc, alpha=0.5)

plt.show()

# Illustris Population

In [ ]:
fname = "./pop-ill_v0.2_e6bc2cb__d20.0_c0.100_mamp+8.845098_mamp1.00/debug_0000.npz"
data = np.load(fname)
print(list(data.keys()))
print(f"Loaded {data['redz'].size:.4e} samples")

In [ ]:
nloud = 5
fobs = data['freq_bins']
hs, fo = holo.sam._strains_from_samples([data[kk] for kk in ['mtot', 'mrat', 'redz', 'fobs']])
colors = holo.plot._get_cmap('plasma')(np.linspace(0.1, 0.9, nloud))

fig, ax = holo.plot.figax(figsize=[12, 8], xlabel='Frequency [yr$^{-1}$]', ylabel='c-Strain')
for ii in holo.utils.tqdm(range(fobs.size-1)):
    fextr = [fobs[ii+jj] for jj in range(2)]
    fextr = np.asarray(fextr)
    cycles = 1.0 / np.diff(np.log(fextr))[0]

    idx = (fextr[0] <= fo) & (fo < fextr[1])
    hs_bin = hs[idx]
    fo_bin = fo[idx]    

    tot = np.sqrt(np.sum(cycles * hs_bin**2))
    ax.plot(fextr*YR, tot * np.ones_like(fextr), 'k--')

    idx = np.argsort(hs_bin)[::-1]
    
    for jj, cc in enumerate(colors):
        if jj >= len(idx):
            break
        hi = idx[jj]
        lo = idx[jj+1:]
        gw_hi = np.sqrt(np.sum(cycles * hs_bin[hi]**2))
        gw_lo = np.sqrt(np.sum(cycles * hs_bin[lo]**2))

        fave = np.average(fo_bin[hi], weights=hs_bin[hi])
        ax.plot(fextr*YR, gw_lo * np.ones_like(fextr), color=cc, lw=0.5)
        ax.scatter(fave*YR, gw_hi, marker='.', color=cc, alpha=0.5)

plt.show()